# Binary classification of sequences into SP or Non-SP using one hot encoding and logistic regression

#### Imports and dependancies

In [22]:
import pandas as  pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score, roc_auc_score


#### Constants

In [23]:
FASTA_PATH = "/Users/jonas/Desktop/Uni/PBL/data/complete_set_unpartitioned.fasta"

#### One hot indexing and encoding

In [ ]:
aas = 'ARNDCEQGHILKMFPSTWYV'
aa2idx = {aa: i for i, aa in enumerate(aas)} # aa 2 index in aas string
num_aa = len(aas)

def one_hot_encode_sequence(seq, max_length):
    # padded with X, which is one hot encoded as a vector containing only zeros
    seq = seq[:max_length].ljust(max_length, 'X')
    
    # Initialize with zeros
    encoding = np.zeros((max_length, num_aa))
    
    for i, aa in enumerate(seq):
        if aa in aa2idx:
            # position in encoding vector = 1 for according aa
            encoding[i, aa2idx[aa]] = 1
    return encoding.flatten()  # makes 1d vector


#### Data loading and prep

In [25]:
def load_and_prep_data(dataPath: str):
    records = []
    with open(dataPath, "r") as f:
        current_record = {}
        for line in f:
            if line.startswith(">"):
                if current_record:
                    records.append(current_record)
                header = line[1:].strip().split("|")
                if len(header) == 3:
                    current_record = {
                        "uniprot_ac": header[0],
                        "kingdom": header[1],
                        "type": header[2],
                        "sequence": ""
                    }
                else:
                    current_record = {}
            elif current_record:
                if not current_record.get("sequence"):
                    current_record["sequence"] = line.strip()
    if current_record:
        records.append(current_record)
    df_raw = pd.DataFrame(records)

    # drop na rows
    df_raw.dropna(subset=['sequence', 'type'], inplace=True)

    # Remove records with 'P' in sequence (if needed)
    df = df_raw[~df_raw["sequence"].str.contains("P")].copy()

    # binary labeling
    df["type"] = df["type"].replace({
        "NO_SP": "0",
        "LIPO": "1",
        "SP": "1",
        "TAT": "1",
        "TATLIPO": "1"
    })

    df_majority = df[df["type"] == "0"]
    df_minority = df[df["type"] == "1"]

    # randomly oversample the data to equalize the NO_SP to SP ratio
    if not df_minority.empty and not df_majority.empty:
        df_minority_upsampled = resample(
            df_minority,
            replace=True,
            n_samples=len(df_majority),
            random_state=42
        )
        df_balanced = pd.concat([df_majority, df_minority_upsampled])
    else:
        df_balanced = df.copy()

    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Total records after oversampling: {len(df_balanced)}")
    print("Class distribution after oversampling:")
    print(df_balanced["type"].value_counts())
    
    """
    # randomly undersample the majority class to match the minority class size
    if not df_minority.empty and not df_majority.empty:
        df_majority_undersampled = resample(
            df_majority,
            replace=False,
            n_samples=len(df_minority),
            random_state=42
        )
        df_balanced = pd.concat([df_majority_undersampled, df_minority])
    else:
        df_balanced = df.copy()

    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Total records after undersampling: {len(df_balanced)}")
    print("Class distribution after undersampling:")
    print(df_balanced["type"].value_counts())
    """

    sequences = df_balanced["sequence"].tolist()
    labels = df_balanced["type"].astype(int).tolist()

    max_length = max(len(seq) for seq in sequences)

    # one hot encoding based on the max length seq in the data
    finSeqs = np.array([one_hot_encode_sequence(seq, max_length) for seq in sequences])
    finLabels = np.array(labels)

    # each part
    train_seqs, test_seqs, train_types, test_types = train_test_split(
        finSeqs, finLabels, test_size=0.2, random_state=42, stratify=finLabels
    )

    print(f"Training set size: {len(train_seqs)}")
    print(f"Test set size: {len(test_seqs)}")

    return train_seqs, test_seqs, train_types, test_types

train_seqs, test_seqs, train_types, test_types = load_and_prep_data(FASTA_PATH)


Total records after oversampling: 1710
Class distribution after oversampling:
type
1    855
0    855
Name: count, dtype: int64
Training set size: 1368
Test set size: 342


#### Model def and training

In [26]:
model = LogisticRegression(max_iter=2000)
model.fit(train_seqs, train_types)

LogisticRegression(max_iter=2000)


#### Model eval

In [27]:
pred_types = model.predict(test_seqs)
print("Accuracy:", round(accuracy_score(test_types, pred_types), 3))

# Precision, Recall, F1
print("Precision:", round(precision_score(test_types, pred_types), 3))
print("Recall:", round(recall_score(test_types, pred_types), 3))
print("F1 Score:", round(f1_score(test_types, pred_types), 3))

# Matthews Correlation Coefficient
print("Matthews Correlation Coefficient:", round(matthews_corrcoef(test_types, pred_types), 3))

if hasattr(model, "predict_proba"):
    proba = model.predict_proba(test_seqs)[:, 1]
    print("ROC AUC:", round(roc_auc_score(test_types, proba), 3))

Accuracy: 0.968
Precision: 0.988
Recall: 0.947
F1 Score: 0.967
Matthews Correlation Coefficient: 0.936
ROC AUC: 0.996
